In [1]:
import numpy as np

import qiskit as qk
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit,Aer
backend = Aer.get_backend('unitary_simulator')

from dreamcoder.domains.quantum_algorithms.primitives import *
from dreamcoder.domains.quantum_algorithms.tasks import *
import dreamcoder as dc
class QiskitTester():
    def __init__(self,unitary_tensor):
        self.unitary_tensor = unitary_tensor
        self.unitary_matrix = tensor_to_mat(unitary_tensor)
        self.n_qubits = get_qubit_number(self.unitary_tensor)
        self.qreg_q = QuantumRegister(self.n_qubits, 'q')
        self.circuit = QuantumCircuit(self.qreg_q)
        
    def q(self,q_num):
        return self.n_qubits -1- q_num
    
    def __enter__(self):
        return self
    
    def __exit__(self,*args, **kwargs):
        self.result = np.array(qk.execute(self.circuit, backend).result().get_unitary()).T
        try:
            np.testing.assert_almost_equal(self.unitary_matrix,self.result, decimal=5)
            print("Code consistent with Qiskit")
        except AssertionError as e:
            print("-----------------------------------")
            print("ERROR: ")
            print(self.unitary_matrix)
            print(self.result)
            print(e)

In [2]:

n_qubit = 2
full_circuit = [n_qubit,
           [["cnot", 0, 1],
           ["swap", 0, 1],
           ["hadamard", 1]]
]

In [3]:
tensor = eye(n_qubit)
tensor_to_mat(swap(cnot(tensor,0,1),0,1))

array([[1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.]])

In [4]:
full_circuit_to_mat(full_circuit)

array([[ 0.70710678,  0.70710678,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.70710678,  0.70710678],
       [ 0.        ,  0.        ,  0.70710678, -0.70710678],
       [ 0.70710678, -0.70710678,  0.        ,  0.        ]])

In [5]:
with QiskitTester(mat_to_tensor(full_circuit_to_mat(full_circuit))) as QT:
    # QT.circuit.h(QT.q(1))
    # QT.circuit.h(QT.q(0))
    # QT.circuit.cnot(QT.q(0), QT.q(1))
    # QT.circuit.h(QT.q(0))
    # QT.circuit.h(QT.q(1))
    QT.circuit.cnot(QT.q(0),QT.q(1))
    QT.circuit.swap(QT.q(0),QT.q(1))
    QT.circuit.h(QT.q(1))
    print(QT.circuit)
QT.result

     ┌───┐   ┌───┐
q_0: ┤ X ├─X─┤ H ├
     └─┬─┘ │ └───┘
q_1: ──■───X──────
                  
Code consistent with Qiskit


array([[ 0.70710678+0.00000000e+00j,  0.70710678+0.00000000e+00j,
         0.        +0.00000000e+00j, -0.        +0.00000000e+00j],
       [ 0.        +0.00000000e+00j, -0.        +0.00000000e+00j,
         0.70710678+0.00000000e+00j,  0.70710678+0.00000000e+00j],
       [ 0.        +0.00000000e+00j, -0.        +0.00000000e+00j,
         0.70710678-8.65956056e-17j, -0.70710678+8.65956056e-17j],
       [ 0.70710678-8.65956056e-17j, -0.70710678+8.65956056e-17j,
         0.        +0.00000000e+00j, -0.        +0.00000000e+00j]])

In [6]:
n_qubit= 3
code = dc.program.Program.parse("(lambda (cnot (minv(mv(no_op $0)))))")
code.infer()
code.evaluate([])(n_qubit)


[[1, -1, 3], [['cnot', 1, 0]]]

In [7]:
state_circuit_to_mat(code.evaluate([])(n_qubit))

array([[1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.]])

In [8]:
tasks = makeTasks()

In [9]:
task = get_task_from_name("hadamard_0",tasks)
code = dc.program.Program.parse("(lambda (h (no_op $0)))")
task.logLikelihood(code),  grammar.logLikelihood(code.infer(), code)


(0.0, -3.8918202981106265)

In [10]:
task =get_task_from_name("cnot_01",tasks)
code = dc.program.Program.parse("(lambda (cnot (no_op $0)))")
task.logLikelihood(code), grammar.logLikelihood(code.infer(), code)

(0.0, -3.8918202981106265)

In [11]:
task = get_task_from_name("cnot_10",tasks)
code = dc.program.Program.parse("(lambda (cnot (minv(mv(no_op $0)))))")
task.logLikelihood(code),  grammar.logLikelihood(code.infer(), code)


(0.0, -7.783640596221253)

In [12]:
task = get_task_from_name("swap_01",tasks)
code = dc.program.Program.parse("(lambda  (cnot(minv(mv_r(cnot(minv (mv (cnot (no_op $0)))))))))")
task.logLikelihood(code),  grammar.logLikelihood(code.infer(), code)


(0.0, -15.567281192442506)

In [13]:
task = get_task_from_name("cz_01",tasks)
code = dc.program.Program.parse("(lambda (h(mv(cnot(mv_r(h (mv (no_op $0))))))))")
task.logLikelihood(code),  grammar.logLikelihood(code.infer(), code)
np.round(state_circuit_to_mat(code.evaluate([])(2)),decimals=2)

array([[ 1., -0.,  0.,  0.],
       [-0.,  1.,  0.,  0.],
       [ 0.,  0.,  1., -0.],
       [ 0.,  0.,  0., -1.]])

In [14]:
with QiskitTester(mat_to_tensor(state_circuit_to_mat(code.evaluate([])(2)))) as QT:
    QT.circuit.cz(QT.q(0),QT.q(1))
    print(QT.circuit)
np.real(np.array(QT.result))

        
q_0: ─■─
      │ 
q_1: ─■─
        
Code consistent with Qiskit


array([[ 1.,  0.,  0., -0.],
       [ 0.,  1.,  0., -0.],
       [ 0.,  0.,  1., -0.],
       [ 0.,  0.,  0., -1.]])

In [15]:
with QiskitTester(mat_to_tensor(state_circuit_to_mat(code.evaluate([])(2)))) as QT:
    QT.circuit.cz(QT.q(1),QT.q(0))
    print(QT.circuit)
np.real(np.array(QT.result))

        
q_0: ─■─
      │ 
q_1: ─■─
        
Code consistent with Qiskit


array([[ 1.,  0.,  0., -0.],
       [ 0.,  1.,  0., -0.],
       [ 0.,  0.,  1., -0.],
       [ 0.,  0.,  0., -1.]])

In [16]:
task = get_task_from_name("cnot_nn_1",tasks)
code = dc.program.Program.parse("(lambda (cnot ((rep (dec(dec(size_to_int $0))) (lambda (mv $0))) (no_op $0))))")
code.evaluate([])(3)
task.logLikelihood(code),  grammar.logLikelihood(code.infer(), code)


(0.0, -14.155496613885283)

In [19]:
%load_ext line_profiler

In [29]:
try:
    import binutil  # required to import from dreamcoder modules
except ModuleNotFoundError:
    import bin.binutil  # alt import if called as module

from dreamcoder.domains.quantum_algorithms.main import main
from dreamcoder.dreamcoder import commandlineArguments
from dreamcoder.utilities import numberOfCPUs

arguments = commandlineArguments(
    featureExtractor=None, # it was TowerCNN
    CPUs=1,#numberOfCPUs(),
    helmholtzRatio=0.5,
    recognitionTimeout=3600,
    iterations=6,
    a=3,
    structurePenalty=1,
    pseudoCounts=10,
    topK=2,
    maximumFrontier=5,
    extras=None,
    solver="python", 
    useRecognitionModel=False,
    enumerationTimeout=500,#-g
    compressor="pypy")   #ocaml, python, pypy  
%lprun -f dc.domains.quantum_algorithms.tasks.QuantumTask.logLikelihood main(arguments)

CUDA is available?: False
using cuda?: False


usage: ipykernel_launcher.py [-h] [--resume RESUME] [-i ITERATIONS]
                             [-t ENUMERATIONTIMEOUT] [-R RECOGNITIONTIMEOUT]
                             [-RS RECOGNITIONSTEPS] [-k TOPK]
                             [-p PSEUDOCOUNTS] [-b AIC] [-l STRUCTUREPENALTY]
                             [-a ARITY] [-c CPUS] [--no-cuda]
                             [-m MAXIMUMFRONTIER] [--reuseRecognition]
                             [--recognition] [--ensembleSize ENSEMBLESIZE]
                             [-g] [-d] [--no-consolidation]
                             [--testingTimeout TESTINGTIMEOUT]
                             [--testEvery TESTEVERY] [--seed SEED]
                             [--activation {relu,sigmoid,tanh}]
                             [--solver {ocaml,pypy,bottom,python}]
                             [-r HELMHOLTZRATIO]
                             [--compressor {pypy,rust,vs,pypy_vs,ocaml,memorize}]
                             [--matrixRank MATRIXRANK] 

*** KeyboardInterrupt exception caught in code being profiled.

Timer unit: 1e-06 s

Total time: 96.3602 s
File: /Users/lsarra/ownCloud/topics/artificial-scientific-discovery/2021_Unitary-Synthesis/ec/dreamcoder/domains/quantum_algorithms/tasks.py
Function: logLikelihood at line 21

Line #      Hits         Time  Per Hit   % Time  Line Contents
    21                                               def logLikelihood(self, e, timeout=None):
    22     85697     896060.0     10.5      0.9          if QuantumTask.last_algorithm != e:
    23      5281       7431.0      1.4      0.0              QuantumTask.last_algorithm = e
    24      5281       5967.0      1.1      0.0              QuantumTask.last_algorithm_evaluations = {}
    25                                           
    26     86057      93750.0      1.1      0.1          for n in range(self.min_size, self.max_size):
    27     86020      73761.0      0.9      0.1              if n not in QuantumTask.last_algorithm_evaluations.keys():
    28      5604     953533.0    170.2      1.0            

In [18]:
code = dc.program.Program.parse("(lambda ((rep (dec(dec(dec 5))) (lambda (mv $0))) (no_op $0)))")
code.evaluate([])(3)
code.infer()
task.logLikelihood(code),  grammar.logLikelihood(code.infer(), code)


(-inf, -13.595880825949859)